# Hybrid Search with LlamaIndex & KDB.AI

Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/offerings/).

KDB.AI hybrid search is a method of similarity search to increase the relevancy of results retrieved from the vector database. It combines two search methods: sparse vector search, and dense vector search.

Sparse vector search uses the BM25 algorithm to find the most relevant keyword matches, while dense vector search finds the most semantically relevant matches.

In KDB.AI, users can run sparse or dense searches independently, or run a hybrid search which combines both sparse and dense vector searches. The results from each search are re-ranked based on user-defined weights for the sparse and dense indexes. The higher the weight for a specific index (sparse or dense), the more influence it will have in determining the final ranking.

## Install dependencies

In [ ]:
%pip install llama-index llama-index-embeddings-huggingface llama-index-llms-openai llama-index-readers-file llama-index-vector-stores-kdbai
%pip install kdbai_client langchain-text-splitters pandas

## Downloading data

**Libraries**

In [1]:
import os
import urllib.request

In [2]:
import nest_asyncio

nest_asyncio.apply()

**Data directories and paths**

In [3]:
# Root path
root_path = os.path.abspath(os.getcwd())

# Data directory and path
data_dir = "data"
data_path = os.path.join(root_path, data_dir)
if not os.path.exists(data_path):
    os.mkdir(data_path)

**Downloading text**

In [4]:
text_url = "https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/hybrid_search/data/inflation.txt"
with urllib.request.urlopen(text_url) as response:
    text_content = response.read().decode("utf-8")

text_file_name = text_url.split('/')[-1]
text_path = os.path.join(data_path, text_file_name)
if not os.path.exists(text_path):
    with open(text_path, 'w') as text_file:
        text_file.write(text_content)

metadata = {
    f"{data_dir}/{text_file_name}": {
        "title": text_file_name,
        "file_path": text_path
    }
}

**Show text data**

In [5]:
def show_text(text_path):
    with open(text_path, 'r') as text_file:
        contents = text_file.read()
    print(contents[:500])
    print("="*80)

In [6]:
show_text(text_path)

 At last year's Jackson Hole symposium, I delivered a brief, direct message. My remarks this year will be a bit longer, but the message is the same: It is the Fed's job to bring inflation down to our 2 percent goal, and we will do so. We have tightened policy significantly over the past year. Although inflation has moved down from its peak—a welcome development—it remains too high. We are prepared to raise rates further if appropriate, and intend to hold policy at a restrictive level until we ar


## KDB.ai Vector Database - session and tables

**Libraries**

In [7]:
import kdbai_client as kdbai
from getpass import getpass

**KDB.AI session**

With the embeddings created, we need to store them in a vector database to enable efficient searching.

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [ ]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You willreceive an email with the required license file and bearer  token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

**KDB.AI table**

In [31]:
# Table - name & schema
table_name = "hs_docs"
table_schema = [
        {"name":"document_id", "type":"bytes"},
        {"name":"text", "type":"bytes"},
        {"name":"embeddings","type":"float32s"},
        {"name":"sparseVectors", "type":"general"},
        {"name":"title", "type":"str"},
        {"name":"file_path", "type":"str"},
        ]

indexeSparse = {
        "name": "sparse_index",
        "type": "bm25",
        "column": "sparseVectors",
        "params": {'k': 1.25, 'b': 0.75},
    }

indexFlat = {
        "name": "flat",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 768, 'metric': 'L2'},
    }

In [11]:
# Connect with kdbai database
db = session.database("default")

In [32]:
# Drop table if exists
try:
    db.table(table_name).drop()
except kdbai.KDBAIException:
    pass

In [33]:
# texts table
table = db.create_table(table_name, table_schema, indexes=[indexFlat, indexeSparse])

## Loading data

In [26]:
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.callbacks import CallbackManager
from llama_index.core import SimpleDirectoryReader

In [27]:
# Helper function - for getting metadata
def get_metadata(file_path):
    return metadata[file_path]

In [28]:
%%time

local_files = [fpath for fpath in metadata]
documents = SimpleDirectoryReader(input_files=local_files, file_metadata=get_metadata)

docs = documents.load_data()
len(docs)

CPU times: user 12.7 ms, sys: 0 ns, total: 12.7 ms
Wall time: 12.9 ms


1

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
EMBEDDING = "sentence-transformers/all-mpnet-base-v2"
embeddings_model = HuggingFaceEmbedding(model_name=EMBEDDING)

### Create vector store, storage context and the index for retrieval, query purposes

In [ ]:
%%time

# Vector Store
text_store = KDBAIVectorStore(table=table, hybrid_search=True)

# Storage context
storage_context = StorageContext.from_defaults(vector_store=text_store)

# Settings
#Settings.callback_manager = callback_manager
Settings.transformations = [SentenceSplitter(chunk_size=500, chunk_overlap=0)]
Settings.embed_model = embeddings_model
Settings.llm = None

# Vector Store Index
index = VectorStoreIndex.from_documents(
    docs,
    use_async=True,
    storage_context=storage_context,
)

In [35]:
table.query()

,document_id,text,embedding,sparseVectors,title,file_path
0,b'c26bcfc2-951e-40bd-959a-ae2b8edd2467',"b'At last year\'s Jackson Hole symposium, I de...","[-0.035284244, 0.0753799, -0.022666411, -0.017...","{101: 1, 2012: 4, 2197: 1, 2095: 3, 1005: 3, 1...",inflation.txt,/content/data/inflation.txt
1,b'e4d97506-7118-49ae-87bf-47c41abe670c',"b""On a 12-month basis, core PCE inflation peak...","[-0.04378559, 0.046354603, -0.030167095, 0.013...","{101: 1, 2006: 7, 1037: 5, 2260: 2, 1011: 6, 3...",inflation.txt,/content/data/inflation.txt
2,b'0014da23-8348-48af-ab56-c64ec48c47cc',b'In the highly interest-sensitive housing sec...,"[-0.07940253, 0.008506958, -0.035946056, -0.00...","{101: 1, 1999: 7, 1996: 21, 3811: 1, 3037: 2, ...",inflation.txt,/content/data/inflation.txt
3,b'1c00e107-b816-40d2-8445-a0ae707c2564',"b""Getting inflation sustainably back down to 2...","[-0.046816133, 0.052543037, -0.038334284, -0.0...","{101: 1, 2893: 1, 14200: 2, 15770: 1, 8231: 1,...",inflation.txt,/content/data/inflation.txt
4,b'73ac6d92-a93f-4b5f-a8c6-8bba94068e3f',b'While nominal wage growth must ultimately sl...,"[-0.033225708, 0.037619803, -0.030979052, -0.0...","{101: 1, 2096: 1, 15087: 2, 11897: 4, 3930: 4,...",inflation.txt,/content/data/inflation.txt
5,b'0decb50d-966f-448f-a2a4-88dacc50a375',b'Doing too little could allow above-target in...,"[-0.042863447, 0.02854309, -0.030805789, -0.03...","{101: 1, 2725: 2, 2205: 2, 2210: 1, 2071: 2, 3...",inflation.txt,/content/data/inflation.txt


## Retrieval from query using Hybrid Search

**Query**

In [36]:
query = '12-month basis'

**Helper function: To display search results**

In [37]:
import pandas as pd

In [38]:
def display_search_results(nodes):
    nodes_df = pd.DataFrame(columns=['score', 'text'])
    for node in nodes:
        nodes_df.loc[len(nodes_df.index)] = (node.score, node.text)
    return nodes_df

**Hybrid Search: Giving equal priority to both sparse and dense vector search**

In [39]:
%%time

retriever = index.as_retriever(
                        vector_store_query_mode="hybrid",
                        similarity_top_k=5,
                        vector_store_kwargs={
                                "index" : "flat",
                                "indexWeight" : 0.5,
                                "indexSparse" : "sparse_index",
                                "indexSparseWeight" : 0.5,
                                },
                            )

CPU times: user 49 μs, sys: 0 ns, total: 49 μs
Wall time: 51 μs


In [40]:
equal_priority_nodes = retriever.retrieve(query)
display_search_results(equal_priority_nodes)

/home/alexg/miniforge/envs/llama_v1.4/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,score,text
0,0.375000,"At last year's Jackson Hole symposium, I deliv..."
1,0.333333,"In my remaining comments, I will focus on core..."
2,0.291667,"Core goods prices fell the past two months, bu..."
3,0.250000,Total hours worked has been flat over the past...
4,0.200000,"Over time, restrictive monetary policy will he..."


**Hybrid Search: Giving more priority to sparse vector search**

In [41]:
%%time

retriever = index.as_retriever(
                        vector_store_query_mode="hybrid",
                        similarity_top_k=5,
                        vector_store_kwargs={
                                "index" : "flat",
                                "indexWeight" : 0.1,
                                "indexSparse" : "sparse_index",
                                "indexSparseWeight" : 0.9,
                                },
                            )

CPU times: user 0 ns, sys: 39 μs, total: 39 μs
Wall time: 41 μs


In [42]:
sparse_priority_nodes = retriever.retrieve(query)
display_search_results(sparse_priority_nodes)

/home/alexg/miniforge/envs/llama_v1.4/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,score,text
0,0.466667,"In my remaining comments, I will focus on core..."
1,0.325000,"Core goods prices fell the past two months, bu..."
2,0.275000,"At last year's Jackson Hole symposium, I deliv..."
3,0.200000,"Over time, restrictive monetary policy will he..."
4,0.183333,Total hours worked has been flat over the past...


**Hybrid Search: Giving more priority to dense vector search**

In [43]:
%%time

retriever = index.as_retriever(
                        vector_store_query_mode="hybrid",
                        similarity_top_k=5,
                        vector_store_kwargs={
                                "index" : "flat",
                                "indexWeight" : 0.9,
                                "indexSparse" : "sparse_index",
                                "indexSparseWeight" : 0.1,
                                },
                            )

CPU times: user 30 μs, sys: 6 μs, total: 36 μs
Wall time: 37.7 μs


In [44]:
dense_priority_nodes = retriever.retrieve(query)
display_search_results(dense_priority_nodes)

/home/alexg/miniforge/envs/llama_v1.4/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,score,text
0,0.475000,"At last year's Jackson Hole symposium, I deliv..."
1,0.316667,Total hours worked has been flat over the past...
2,0.258333,"Core goods prices fell the past two months, bu..."
3,0.200000,"Over time, restrictive monetary policy will he..."
4,0.200000,"In my remaining comments, I will focus on core..."


**Conclusion**
- In the sparse search results, we can see the terms we are interested directly i.e "12-month basis" rather than terms having similar meanings.
- In the dense search resutls, we can see the most related or similar text to the query.

## Delete the KDB.AI Table
Once finished with the table, it is best practice to drop it.

In [45]:
table.drop()